---
디시전 트리로 추천/비추천 예측
방법 1. User마다 decision tree 모델을 하나씩 갖는다.
방법 2. user각자 고려하지 않고, user_id도 feature로 사용한다

X - games features
Y - 추천 여부

In [4]:
# 데이터셋 로드 및 불필요한 column 제거
import numpy as np
import pandas as pd

# Load the data
data = pd.read_csv('../datasets/merged_steam_games_.csv')

# convert date_realese columns to unixtime
data['date_release'] = pd.to_datetime(data['date_release'])
data['date_release'] = data['date_release'].apply(lambda x: int(x.timestamp()))

data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].apply(lambda x: int(x.timestamp()))

ids = pd.DataFrame(data['app_id'])
# data = data.drop(['app_id'], axis=1)

# 전처리 안된 column 제거
data = data.drop(['categories'], axis=1)

data = data.drop(['title'], axis=1)
data = data.drop(['release_date'], axis=1)  

print(data.shape)
print(data.head())
print(data.columns)
print(data.dtypes)
print(data.describe())


(46365, 56)
   app_id  date_release  win  mac  linux  rating  positive_ratio  \
0  552520    1522022400    1    0      0       7              80   
1  552520    1522022400    1    0      0       7              80   
2  552520    1522022400    1    0      0       7              80   
3  552520    1522022400    1    0      0       7              80   
4  552520    1522022400    1    0      0       7              80   

   user_reviews  price_final  price_original  ...  RPG  Indie  \
0        129943         60.0             0.0  ...    0      0   
1        129943         60.0             0.0  ...    0      0   
2        129943         60.0             0.0  ...    0      0   
3        129943         60.0             0.0  ...    0      0   
4        129943         60.0             0.0  ...    0      0   

   Software Training  Simulation  Game Development  Massively Multiplayer  \
0                  0           0                 0                      0   
1                  0           0  

In [5]:
# 데이터셋에서 게임 데이터 분할
recommend_columns = ['app_id','helpful', 'funny', 'date', 'is_recommended', 'hours',
       'user_id']

recommend_datas = data[recommend_columns]

game_infos = data.drop(recommend_columns[1:], axis=1)

## Scailing

In [6]:
from sklearn.preprocessing import StandardScaler
scaled_data = game_infos.copy()
scale_cols = ['date_release','positive_ratio','user_reviews','price_final','price_original', 'required_age', 'achievements', 'positive_ratings', 'negative_ratings', 
              'average_playtime', 'median_playtime', 'owners', 'price',]
# scale_cols = [col for col in scaled_data.columns if col not in no_scale_cols]
unique_values = ids['app_id'].unique()
print(unique_values)

index_positions = [ids[ids['app_id'] == value].index[0] for value in unique_values]

scaled_data = game_infos.iloc[index_positions]

# Scale the data
scaler = StandardScaler()
scaled_data[scale_cols] = scaler.fit_transform(scaled_data[scale_cols])

[552520 242760 444200 255710    220 594650 582010    550 271590 294100
 107410 292030 105600 438100 252490    570 377160  22380 239140 812140
 632360 221100 270880 251570 289070 238960 108600 548430 367520    730
 264710 646570 275850 230410 413150 359550 218620 435150    620 648800
 518790 444090 960090   4000 489830 381210 311210 374320  12210    240
    440  48700  49520 203160 227300 236390 262060 284160 291550 304930
 306130 319630 322330 346110 379720 391540 394360 578080 431960 427520
 814380 945360 477160 250900]


/var/folders/tr/1pblcvrd48j5lfn0khj_hm4w0000gn/T/ipykernel_93633/3791647812.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data[scale_cols] = scaler.fit_transform(scaled_data[scale_cols])


In [7]:
# 스케일 된 게임 데이터와 추천 데이터 다시 합치기
merged_data = pd.merge(recommend_datas, scaled_data, on='app_id')
print(merged_data.shape)

(46365, 56)


## 테스트 데이터 분리

In [8]:
from sklearn.model_selection import train_test_split

Y = merged_data['is_recommended']
X = merged_data.drop(['is_recommended'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=156)

X_colums = merged_data.columns.drop(['is_recommended'])
df_X_train = pd.DataFrame(X_train , columns=X_colums)
df_Y_train = pd.DataFrame(Y_train, columns=['is_recommended'])
df_X_test = pd.DataFrame(X_test, columns=X_colums)
df_Y_test = pd.DataFrame(Y_test, columns=['is_recommended'])

In [10]:

from sklearn.tree import DecisionTreeClassifier

# 특정 유저 아이디에 대한 추천 게임 예측
# 각 유저별로 decision tree를 만들어서 예측

# 유저 아이디를 입력받아서 해당 유저의 추천 데이터만 추출
target_user_id = 7606333

train_indices = df_X_train.index[df_X_train['user_id'] == target_user_id].tolist()
X_target_train = df_X_train.loc[train_indices]
Y_target_train = df_Y_train.loc[train_indices]

test_indices = df_X_test.index[df_X_test['user_id'] == target_user_id].tolist()
X_target_test = df_X_test.loc[test_indices]
Y_target_test = df_Y_test.loc[test_indices]

# 모델 학습
model = DecisionTreeClassifier(max_depth=5, random_state=0)
model.fit(X_target_train, Y_target_train)

#예측
print(model.predict(X_target_test))
print(model.score(X_target_train, Y_target_train))

[1]
1.0
